# Objective : Identify topics or tags/labels from news articles and map to prospective sales vertical in a prescriptive way.¶

# News classification 

Author : 
Praveen Vijayan ,
TCS A&I - Pune

# Importing libraries

In [53]:
from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
import csv
#import nltk
import string
import re
import sys
from sklearn import metrics

In [21]:
def find_news_category(x) :
    class_output_NB = clf_NB.predict(x)
    class_description = " "
    if class_output_NB == '1':
        class_description = "accounting"
    if class_output_NB == '2':
        class_description = "aero"
    if class_output_NB == '3':
        class_description = "bfsi"
    if class_output_NB == '4':
        class_description = "oil&gas"        
    print("@NB : This NEWS is related to : " + class_description)        
    class_output_svm = clf_svm.predict(x)
    class_description = " "
    if class_output_NB == '1':
        class_description = "accounting"
    if class_output_NB == '2':
        class_description = "aero"
    if class_output_NB == '3':
        class_description = "bfsi"
    if class_output_NB == '4':
        class_description = "oil&gas"       
    print("@SVM : This NEWS is related to : " + class_description)       

# Reading the dataset

In [22]:
# read CSV file & load into list
with open("sub_News_data_train.csv",'r') as my_file:
    reader = csv.reader(my_file, delimiter=',')
    news_data_train = list(reader)
    
with open("sub_News_data_test.csv",'r') as my_file:
    reader = csv.reader(my_file, delimiter=',')
    news_data_test = list(reader)   

# Exploring the data 

In [23]:
print(news_data_train[3])

['1', 'Nortel shares down following news of RCMP probe', 'TORONTO - Shares of Nortel Networks dipped on the TSX in the wake of news the RCMP has launched an investigation into accounting practices at the telecommunications company. ']


In [24]:
print("class label : " + news_data_train[3][0])

class label : 1


In [25]:
print("Heading : " + news_data_train[3][1])

Heading : Nortel shares down following news of RCMP probe


In [26]:
print("Content : " +news_data_train[3][2])

Content : TORONTO - Shares of Nortel Networks dipped on the TSX in the wake of news the RCMP has launched an investigation into accounting practices at the telecommunications company. 


# Bucketing the data 

In [27]:
stopwords_file = open("stopwords.txt", "r" , encoding="utf8") 
stopwords = stopwords_file.read()
#stopwords.split()

In [28]:
#stop = set(stopwords.words('english'))

In [29]:
STOPWORD_customized = {'will','new','york','will' ,'quot','year ','company ','week', 'one','two','three','reuters','reuters' , 'monday','tuesday','wednesday','thursday','friday','saturday','sunday','yesterday'}

In [30]:
label = [] # class
text_doc = [] # news text 
c =0 # to control count of 
for i in news_data_train:
    c= c+1
    if c < 50000:
        label.append(i[0])            
        merged = i[1]+i[2] # combining header and content 
        lowers = merged.lower()   # to lower      
        no_punctuation = re.sub(r'[^\w\s]',' ', lowers)    # remove punctuation
        temp = ' '.join(word for word in no_punctuation.split() if len(word)>3) # remove samll words
        filtered = ' '.join(word for word in temp.split() if word not in stopwords.split()) # remove stopwords
        filtered = ' '.join(word for word in filtered.split() if word not in STOPWORD_customized) # remove stopwords
        text_doc.append(filtered)

In [31]:
import codecs
with codecs.open("parsed_NLP_text.txt", 'w', encoding='utf_8') as f: 
    for sent in text_doc:
        f.write(sent +'. '+ '\n')

In [32]:
print(text_doc[4])

investors nortel jobsottawa nortel networks corp investors predicted telecom equipment giant slash jobs reports long overdue results shrugged news criminal probe high profile accounting woes


In [33]:
#print(text_doc)
text = ""
for i in range (1,len(text_doc)):
    temp = (text_doc[i])
    text = text + temp

In [34]:
from collections import Counter
word_count = Counter()
word_count.update((word for word in text.split()))
#for word, count in word_count.most_common():
    #print(word, ":" ,count,"\n")      

In [35]:
sys.getdefaultencoding()

'utf-8'

In [36]:
from collections import Counter
#Counter(label).keys() # equals to list(set(words))

In [37]:
#Counter(label).values() # counts the elements' frequency

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
vectorizer = CountVectorizer()
corpus = text_doc  # processed text corpous 
X = vectorizer.fit_transform(corpus)
X.shape

(2637, 7482)

In [40]:
#from sklearn.feature_extraction.text import TfidfTransformer
#tf_transformer = TfidfTransformer(use_idf=True).fit(X)
#X_tf = tf_transformer.transform(X)
#X_tf.shape

In [41]:
tfvectorizer = TfidfVectorizer(max_df=0.5, max_features= 10000, min_df=2, stop_words='english',use_idf= True)
X_tfidf = tfvectorizer.fit_transform(corpus)
X_tfidf

<2637x3701 sparse matrix of type '<class 'numpy.float64'>'
	with 48012 stored elements in Compressed Sparse Row format>

In [45]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

# Classification (NB)

In [46]:
# X -> full corpus
# X_tfidf -> TFIDF reduced 

In [57]:
# X  has text data as martrix
y = label
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB()

clf = MultinomialNB()


In [58]:
clf_NB.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [59]:
# make predictions
expected = label
predicted = clf_NB.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          1       0.99      0.99      0.99       463
          2       0.97      0.99      0.98       222
          3       0.99      0.98      0.98       619
          4       0.99      1.00      1.00      1333

avg / total       0.99      0.99      0.99      2637

[[ 460    1    1    1]
 [   0  220    1    1]
 [   4    6  604    5]
 [   1    0    2 1330]]


In [60]:
clf_NB.fit(X_tfidf, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [61]:
# make predictions
expected = label
predicted = clf_NB.predict(X_tfidf)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          1       0.98      0.97      0.98       463
          2       0.99      0.86      0.92       222
          3       0.95      0.95      0.95       619
          4       0.97      1.00      0.98      1333

avg / total       0.97      0.97      0.97      2637

[[ 449    1    5    8]
 [   1  190   24    7]
 [   5    1  585   28]
 [   1    0    1 1331]]


# Classification (svm)

In [62]:
from sklearn import svm
clf_svm = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [63]:
clf_svm.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [64]:
# make predictions
expected = label
predicted = clf_svm.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00       463
          2       1.00      1.00      1.00       222
          3       1.00      1.00      1.00       619
          4       1.00      1.00      1.00      1333

avg / total       1.00      1.00      1.00      2637

[[ 463    0    0    0]
 [   0  222    0    0]
 [   0    0  619    0]
 [   0    0    0 1333]]


In [65]:
clf_svm.fit(X_tfidf, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [66]:
#clf.predict(x)

In [67]:
# make predictions
expected = label
predicted = clf_svm.predict(X_tfidf)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00       463
          2       1.00      1.00      1.00       222
          3       1.00      1.00      1.00       619
          4       1.00      1.00      1.00      1333

avg / total       1.00      1.00      1.00      2637

[[ 463    0    0    0]
 [   0  222    0    0]
 [   0    0  618    1]
 [   1    0    0 1332]]


# Testing the model 

# Accuracy on training set

In [144]:
import random
random_news_generator = random.randint(1,len(news_data_test))

In [145]:
news_data_test[random_news_generator][2]

'Continuing to snap up branch networks in states with large Hispanic populations, Citigroup Inc. Tuesday, Aug. 24, agreed to buy First American Bank of Texas for an undisclosed price.'

In [146]:
new_text_doc = []

In [147]:
new_text = news_data_test[random_news_generator][2]

In [148]:
#new_text = ["AFP - The world economy, powered in part by China, will grow this year at its briskest pace in almost three decades -- five percent -- before losing steam in 2005 in the face of higher oil prices, the IMF said."]

In [149]:
new_text_doc.append(new_text)

In [151]:
x = tfvectorizer.transform(new_text_doc).toarray()

In [152]:
find_news_category(x)

@NB : This NEWS is related to : bfsi
@SVM : This NEWS is related to : bfsi
